In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random
from tqdm import tqdm
import re
import itertools
from wordle_helpers import wordle_scoring

In [2]:
col_names =\
    pd.read_csv("./data/top_100_freq_used_5_letter_words_vs_webster_dict_all_five_letter_words.csv.gz", nrows=0).columns
types_dict = {"sum_first_pos_score": np.int8,
              "sum_second_pos_score": np.int8,
              "sum_correct_letter_pos_score": np.int8,}
types_dict.update({col: "string[pyarrow]" for col in col_names if col not in types_dict})
scoring = pd.read_csv("./data/top_100_freq_used_5_letter_words_vs_webster_dict_all_five_letter_words.csv.gz",
                      dtype=types_dict,
                      engine="pyarrow",
                      compression="gzip")

In [3]:
scoring.head(5)

,challenge_word,guess_pair,first_guess,second_guess,first_pos_score,second_pos_score,sum_first_pos_score,sum_second_pos_score,correct_letter_pos_score,sum_correct_letter_pos_score
0,ficus,"place, tough",place,tough,-1 -1 -1 0 -1,-1 -1 0 -1 -1,-4,-4,-1 -1 0 0 -1,-3
1,ficus,"sound, threw",sound,threw,0 -1 0 -1 -1,-1 -1 -1 -1 -1,-3,-5,0 -1 0 -1 -1,-3
2,ficus,"front, magic",front,magic,1 -1 -1 -1 -1,-1 -1 -1 0 0,-3,-3,1 -1 -1 0 0,-1
3,ficus,"could, agent",could,agent,0 -1 0 -1 -1,-1 -1 -1 -1 -1,-3,-5,0 -1 0 -1 -1,-3
4,ficus,"front, music",front,music,1 -1 -1 -1 -1,-1 0 0 0 0,-3,-1,1 0 0 0 0,1


In [4]:
scoring.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945472 entries, 0 to 2945471
Data columns (total 10 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   challenge_word                string
 1   guess_pair                    string
 2   first_guess                   string
 3   second_guess                  string
 4   first_pos_score               string
 5   second_pos_score              string
 6   sum_first_pos_score           int8  
 7   sum_second_pos_score          int8  
 8   correct_letter_pos_score      string
 9   sum_correct_letter_pos_score  int8  
dtypes: int8(3), string(7)
memory usage: 278.8 MB


# best word pair

**The best `word pair` is the one that has the `highest score`**

In [5]:
guess_pair_sum_scores = (scoring
                        .groupby(by="guess_pair")[["sum_correct_letter_pos_score"]].sum()
                        .sort_values(by="sum_correct_letter_pos_score", ascending=False))

guess_pair_sum_scores.rename(columns={
    "sum_correct_letter_pos_score": "overall_sum_correct_letter_pos_score"}, inplace=True)
guess_pair_sum_scores[:10]

,overall_sum_correct_letter_pos_score
guess_pair,
"close, train",-7655.0
"story, plane",-7828.0
"point, share",-7898.0
"since, party",-8031.0
"share, count",-8051.0
"point, scare",-8057.0
"short, plane",-8209.0
"place, story",-8332.0
"train, chose",-8395.0


In [6]:
best_guess_pair = guess_pair_sum_scores.idxmax()[0]
best_guess_pair

'close, train'

In [7]:
worst_guess_pair = guess_pair_sum_scores.idxmin()[0]
worst_guess_pair

'might, found'

In [8]:
best_guess_pair_df = \
    (scoring
     .loc[scoring["guess_pair"] == guess_pair_sum_scores.idxmax()[0]])
best_guess_pair_df.head()

,challenge_word,guess_pair,first_guess,second_guess,first_pos_score,second_pos_score,sum_first_pos_score,sum_second_pos_score,correct_letter_pos_score,sum_correct_letter_pos_score
419,ficus,"close, train",close,train,0 -1 -1 0 -1,-1 -1 -1 0 -1,-3,-4,0 -1 -1 0 -1,-3
948,budgy,"close, train",close,train,-1 -1 -1 -1 -1,-1 -1 -1 -1 -1,-5,-5,-1 -1 -1 -1 -1,-5
1477,litho,"close, train",close,train,-1 0 0 -1 -1,0 -1 -1 0 -1,-3,-3,0 0 0 0 -1,-1
2006,pence,"close, train",close,train,0 -1 -1 -1 1,-1 -1 -1 -1 0,-2,-4,0 -1 -1 -1 1,-2
2535,loord,"close, train",close,train,-1 0 1 -1 -1,-1 0 -1 -1 -1,-2,-4,-1 0 1 -1 -1,-2


# conclusion  

My best 2 word opener is `"close, train"`, which coincidentally are anagrams of the ones identified by [3blue1brown](https://www.youtube.com/watch?v=fRed0Xmc2Wg) - timestamp 10:52

## anagrams  

anagrams of `close, train`:  
- `soare, clint`
- `slane, troic` (slane - a spade for cutting turf, troic - pertaining to Troy)
- `lanes, troic`
- `salet, orcin` (salet - a combat helmet, orcin - a colorless substance obtained from certain lichens)
- `tales, orcin`
- `lates, orcin`
- `slate, orcin`
- `trace, loins`
- `trace, lions`
- `crate, lions`
- `crate, loins`

`arose, clint` maybe more appropriate as `soare` (a young hawk) is rarely used.  

additional anagrams:  
- `arise, clton`
- `raise, clton`

the second words in additional anagrams above are the left over letters